In [7]:
import numpy as np
from scipy.signal import convolve2d 
from os import path, getcwd, system, getenv
from astropy.io import fits
from time import perf_counter
import urllib.request
import os

import matplotlib
import math
import matplotlib.pyplot as plt
from matplotlib import cm
import glob
import matplotlib.gridspec as gridspec


from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.backends.backend_pdf import PdfPages

import sys,inspect
currentdir = path.dirname(path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = path.dirname(currentdir)
sys.path.append(parentdir)
# from deconv import cube_deconv
# from ppxf_wrap import ppxf_wrap

# import warnings

# if True:
#     print('Set warnings.filterwarnings as "ignore"')
#     print('Warning Message will not be printed')
#     warnings.filterwarnings("ignore")

from astropy.convolution import convolve, Gaussian1DKernel, Box1DKernel
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
from scipy import constants
import pandas as pd

# import plotly.express as px
from astroquery.mast import Observations


from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u
from astropy.nddata.utils import Cutout2D

from astropy.wcs import WCS
import astropy

from urllib.error import HTTPError
from astroquery.skyview import SkyView
import pickle



In [44]:
computer =1
maindir = "C:/Users/maxri/Desktop/Classes_4-1/Research/fuse"
obn = 0
    

t2 = pd.DataFrame([])
for ii in range(80,81,1):
    obn = ii
    #1
    #Create Object List/
    #Save to Object Tables
    
    skydowni ="./images_redOb{}".format(obn); 
    catalogdir = "C:/Users/maxri/Desktop/Classes_4-1/Research/data/VI_129_fuse.dat.fits"
    catalog = fits.open(catalogdir)

    catheader = catalog[0]; catlist = catalog[1];

    dflist = pd.DataFrame(catlist.data)



    catalog.close()


    obs=dflist.query("Class == {} and Avail=='y'".format(obn));
    
    if len(obs) == 0:
        continue

    dflist

#     #Reading Index
#     indexdir = f"{maindir}/tbls/{obn}index.txt"
#     indexdir

#     ipd = pd.read_table(indexdir, header = None)




    obsn1 = []
    for i in obs.iterrows():
        b = [i[1][1].split()[0],i[1][2], i[1][3]] 
        obsn1.append(b)



    obsn1

    s1 = maindir+"/tbls/Objects{}.tbl".format(obn); 
    
    stest = maindir + "/targets.txt"
    np.savetxt(s1,obsn1,fmt="%s",delimiter = " ")



    #2
    #Skyview Query by Object name or Coordinate, reads in the from Object table. 
    #Image query block

    size = 1600

    width_val = u.Quantity(size, unit = 'arcsec')
    height_val =  u.Quantity(size, unit = 'arcsec')

    t3 = pd.DataFrame([])
    # if computer == 2:
    #     t = pd.read_table("C:/Users/maxri/OneDrive/Desktop/Classes 4-1/Research/fuse/objs2.tbl",header=None)
    # if computer == 1:
    #     t = pd.read_table("C:/Users/maxri/Desktop/Classes 4-1/Research/fuse/tbls/objs2.tbl",header=None)


        #Read selected class object table
    if computer == 1:
        try:
            t2 = pd.read_table(s1,header=None)
        except:
            continue
    




    # for i in t.values:
    #     t3.append(i[0])


    coordsn = [];t4=[]


    for i in t2.values:
        b = i[0].split()
        c = SkyCoord(b[1],b[2], unit=u.deg) #commented out for name only
        coordsn.append(c)
        t4.append(b[0])

    coordsn


    objnamesn = t4; objnames = objnamesn;


    obstables2 = []; coords = [];

    #pixels initially 300 x 300
    #Initial Skyview Query by name

    surveys = ['DSS', 'DSS1 Blue', 'DSS1 Red', 'DSS2 Red', 'DSS2 Blue', 'DSS2 IR'] #use surveyindex = 3 Dss2red

    surveyindex = 3


    for i in range(len(objnamesn)):
            paths = [];
            try:
                paths = SkyView.get_images(position = coordsn[i], 
                           survey=surveys[surveyindex],width = width_val,height = height_val,pixels = [1000,1000])
                hdr = paths[0][0].header
                a = hdr['HISTORY'][33].split()
                cd= [float(a[2]),float(a[3])]
                print(objnamesn[i],surveys[surveyindex])
                c = SkyCoord(cd[0],cd[1], unit=u.deg)
                coords.append(c)

            except HTTPError:
                print("No Result for ", objnamesn[i], surveys[surveyindex])
            except TypeError:
                print("No Result for ", objnamesn[i], surveys[surveyindex])
            except IndexError:
                print("No Result for ", objnamesn[i], surveys[surveyindex])
            obstables2.append(paths)


    objnames = objnamesn

    result = pd.DataFrame(obstables2)
    obstables2, result, objnamesn,coords
    #objnamesn, coordsn


    #3
    #Skyview Download Images Code Block, to same convention as before
    #Create Download directory too

    pt = os.path.join(maindir, skydowni[2:])
    try:
        os.mkdir(os.path.normpath(pt))
        print("Created images_red directory")
    except:
        print("Directory already exists")

    result;
    coords
    for i in range(len(result)):
        a = surveys[surveyindex].split() #Survey
        b = objnames[i].split()
        name = b[0] #+ b[1]     #OBjName
        if i < 9:
            fname = "000{}_{}_{}_{}_{}".format(i+1,coordsn[i].ra.deg,coordsn[i].dec.deg,a[0]+a[1],name)
        else:
            fname = "00{}_{}_{}_{}_{}".format(i+1,coordsn[i].ra.deg,coordsn[i].dec.deg,a[0]+a[1],name)
        #print(fname)
        try:
            result[0][i][0].writeto(skydowni+ "/"+fname+".fits", output_verify='exception', overwrite=True, checksum=False)
        except:
            print(f"No result for {fname}")


  

    #Download Method
    # obs_tables=[]
    # for i in range(len(objnames)):
    #     print(objnames[i]) #objectname=objname[i] vs coordinates=coordsn[i] Object Name vs Coordinate Query
    #     obs_tables.append(Observations.query_criteria(obs_collection='FUSE',coordinates=coordsn[i], radius=".5 deg"))


    #Pickle Method obs_tables
    pickledir = maindir + "/Pickles/" + "Ob{}".format(obn)
    with open(pickledir, 'rb') as f:
        # The protocol version used is detected automatically, so we do not
        # have to specify it.
        obs_tables = pickle.load(f)
        f.close

    #Pickle Method Product List
    pickledir2 = maindir + "/Pickles/" + "pl{}".format(obn)
    with open(pickledir2, 'rb') as g:
        # The protocol version used is detected automatically, so we do not
        # have to specify it.
        pl = pickle.load(g)
        g.close

    #Main Fuse Download
    fnames=[]


    #download files
    if computer == 1:
        down_dir=maindir + "/temp"
    

    #Download block 2
    for i in range(len(obs_tables)):
    # for i in [6]:
        otb=obs_tables[i]
        urls=pl[i]['dataURI']
        rr=[x.find('1attagfraw') for x in urls]
        dpl=pl[i][np.array(rr)>0]
        print(urls)
        
        stopper = 0
        try:
            manifest = Observations.download_products(dpl, download_dir=down_dir)
        except Warning:
            stopper = stopper +1
        print(obs_tables[i]['target_name'])
        print(manifest)

       
        if stopper == 1:
            continue
        print(obs_tables[i]['target_name'])
        print(manifest)
    #    needs documentation

        exptimes=obs_tables[0]['t_exptime']
    #     print(exptimes)
        cond=exptimes>5000
        md = manifest["Local Path"]
        t1 = pd.DataFrame(md)
        t3 = pd.concat([t3,t1],ignore_index=True)


    #5
    # Pointings download/Creating organised Data Frames
    # Slowest Block currently
    # Not sure if some parts could be removed.


   
       




  

    #6
    #Max & Mean exposure calc


   


    # Table: 
    # Galaxy, Program ID, N pointings (exp >5000s), RA, DEC, Max Exp, Mean Exp
    # *note on aperture
    # *RA DEC unit check





NGC4631-A DSS2 Red
NGC4631-B DSS2 Red
NGC891-1 DSS2 Red
NGC891-2 DSS2 Red
NGC891-3 DSS2 Red
NGC7714 DSS2 Red
NGC7714 DSS2 Red
NGC7714 DSS2 Red
NGC7714 DSS2 Red
NGC7714 DSS2 Red
NGC5457-POS1 DSS2 Red
NGC5194-POS1 DSS2 Red
NGC5194-POS4 DSS2 Red
NGC5194-POS2 DSS2 Red
NGC5194-POS3 DSS2 Red
NGC5194-POS5 DSS2 Red
NGC5055-POS2 DSS2 Red
NGC5055-POS1 DSS2 Red
NGC4631-I DSS2 Red
NGC4736-POS1 DSS2 Red
NGC4631-A DSS2 Red
NGC224 DSS2 Red
NGC224 DSS2 Red
NGC224 DSS2 Red
NGC224 DSS2 Red
NGC5457-POS2 DSS2 Red
NGC4736-POS2 DSS2 Red
NGC4631-H DSS2 Red
NGC4631-F DSS2 Red
NGC1313 DSS2 Red
NGC7714 DSS2 Red
NGC5055 DSS2 Red
NGC2403 DSS2 Red
NGC5866 DSS2 Red
Directory already exists
No result for 0001_190.537_32.5767_DSS2Red_NGC4631-A
No result for 0002_190.537_32.56_DSS2Red_NGC4631-B
No result for 0003_35.6208_42.3533_DSS2Red_NGC891-1
No result for 0004_35.6667_42.3767_DSS2Red_NGC891-2
No result for 0005_35.6867_42.37_DSS2Red_NGC891-3
No result for 0006_354.059_2.1553_DSS2Red_NGC7714
No result for 0007_354.

INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\c0570202000\c05702020131attagfraw.fit.gz with expected size 106945. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\c0570202000\c05702020141attagfraw.fit.gz with expected size 206703. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\c0570202000\c05702020151attagfraw.fit.gz with expected size 292577. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\c0570202000\c05702020161attagfraw.fit.gz with expected size 317342. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\c0570202000\c05702020171attagfraw.fit.gz with expected size 296488. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDow

INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\c0570101000\c05701010011attagfraw.fit.gz with expected size 175947. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\c0570101000\c05701010021attagfraw.fit.gz with expected size 178916. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\c0570101000\c05701010031attagfraw.fit.gz with expected size 172534. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\c0570201000\c05702010011attagfraw.fit.gz with expected size 212492. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\c0570201000\c05702010021attagfraw.fit.gz with expected size 224916. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDow

INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\p1340101000\p13401010071attagfraw.fit.gz with expected size 296296. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\p1340101000\p13401010081attagfraw.fit.gz with expected size 258718. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\p1340101000\p13401010091attagfraw.fit.gz with expected size 259785. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\p1340101000\p13401010101attagfraw.fit.gz with expected size 260002. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\p1340101000\p13401010111attagfraw.fit.gz with expected size 233917. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDow

INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\b1140101000\b11401010011attagfraw.fit.gz with expected size 127097. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\b1140101000\b11401010021attagfraw.fit.gz with expected size 129989. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\b1140101000\b11401010031attagfraw.fit.gz with expected size 103960. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\b1140101000\b11401010041attagfraw.fit.gz with expected size 86894. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\b1140101000\b11401010051attagfraw.fit.gz with expected size 208854. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDown

...

INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\c0370204000\c03702040061attagfraw.fit.gz with expected size 272730. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\c0370204000\c03702040071attagfraw.fit.gz with expected size 270749. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\c0370204000\c03702040081attagfraw.fit.gz with expected size 264903. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\c0370204000\c03702040091attagfraw.fit.gz with expected size 277467. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\c0370204000\c03702040101attagfraw.fit.gz with expected size 282910. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDow

INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\b1060801000\b10608010011attagfraw.fit.gz with expected size 156969. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\b1060801000\b10608010021attagfraw.fit.gz with expected size 150239. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\b1060801000\b10608010031attagfraw.fit.gz with expected size 135400. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\b1060801000\b10608010041attagfraw.fit.gz with expected size 68133. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\b1060801000\b10608010051attagfraw.fit.gz with expected size 139780. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDown

INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\f9180202000\f91802020011attagfraw.fit.gz with expected size 1056743. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\f9180202000\f91802020021attagfraw.fit.gz with expected size 768638. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\f9180202000\f91802020031attagfraw.fit.gz with expected size 351817. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\f9180202000\f91802020041attagfraw.fit.gz with expected size 393796. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\f9180202000\f91802020051attagfraw.fit.gz with expected size 489279. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDo

INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\f9180602000\f91806020091attagfraw.fit.gz with expected size 611184. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\f9180602000\f91806020101attagfraw.fit.gz with expected size 1008006. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\f9190901000\f91909010021attagfraw.fit.gz with expected size 213616. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\f9190901000\f91909010031attagfraw.fit.gz with expected size 717172. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\f9190901000\f91909010041attagfraw.fit.gz with expected size 132685. [astroquery.query]
 target_name 
-------------
NGC2403-HK542
  NGC2403-VS9
NGC2403-HK361
NGC2403-HK270


INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\f9191601000\f91916010011attagfraw.fit.gz with expected size 995112. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\f9191601000\f91916010021attagfraw.fit.gz with expected size 863443. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\f9191601000\f91916010031attagfraw.fit.gz with expected size 604283. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\f9191601000\f91916010041attagfraw.fit.gz with expected size 887868. [astroquery.query]
INFO: Found cached file C:/Users/maxri/Desktop/Classes_4-1/Research/fuse/temp/mastDownload\FUSE\f9191601000\f91916010051attagfraw.fit.gz with expected size 356936. [astroquery.query]
target_name
-----------
    NGC5866
                                                 

In [70]:
t3["Local Path"]

0       C:/Users/maxri/Desktop/Classes_4-1/Research/fu...
1       C:/Users/maxri/Desktop/Classes_4-1/Research/fu...
2       C:/Users/maxri/Desktop/Classes_4-1/Research/fu...
3       C:/Users/maxri/Desktop/Classes_4-1/Research/fu...
4       C:/Users/maxri/Desktop/Classes_4-1/Research/fu...
                              ...                        
2319    C:/Users/maxri/Desktop/Classes_4-1/Research/fu...
2320    C:/Users/maxri/Desktop/Classes_4-1/Research/fu...
2321    C:/Users/maxri/Desktop/Classes_4-1/Research/fu...
2322    C:/Users/maxri/Desktop/Classes_4-1/Research/fu...
2323    C:/Users/maxri/Desktop/Classes_4-1/Research/fu...
Name: Local Path, Length: 2324, dtype: object

In [62]:
len(t2)

34

In [69]:
counter = 0
for j in range(0,len(obs_tables)):
    counter += len(obs_tables[j])
counter

215

In [82]:
obs_tables[1]

dataproduct_type,calib_level,obs_collection,obs_id,target_name,s_ra,s_dec,t_min,t_max,t_exptime,wavelength_region,filters,em_min,em_max,target_classification,obs_title,t_obs_release,instrument_name,proposal_pi,proposal_id,proposal_type,project,sequence_number,provenance_name,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,intentType,obsid,objID,objID1,distance
str8,int64,str4,str11,str9,float64,float64,float64,float64,float64,str2,str1,float64,float64,str1,str53,float64,str3,str6,str4,str1,str1,int64,str1,str56,str83,str78,str6,bool,float64,str7,str6,str6,str6,float64
spectrum,2,FUSE,c0570201000,NGC4631-F,190.57500000000005,32.563333,52695.0706713,52695.51438938,16500.0,UV,--,90000000000.0,119000000000.0,--,Observations of O VI Emission in the Halo of NGC 4631,nan,FUV,Murphy,C057,--,--,--,--,CIRCLE ICRS 190.575 32.563333 0.00416666666667,http://archive.stsci.edu/browse/previews/fuse/c0570201/c057020100000specttagf.gif,http://archive.stsci.edu/pub/vospectra/fuse2/c057020100000nvo4ttagfcal_vo.fits,PUBLIC,--,5885.0,science,342900,569778,569778,101.01597069616007
spectrum,2,FUSE,c0570202000,NGC4631-F,190.57500000000005,32.563333,53087.42503472,53088.28412073,27762.0,UV,--,90000000000.0,119000000000.0,--,Observations of O VI Emission in the Halo of NGC 4631,nan,FUV,Murphy,C057,--,--,--,--,CIRCLE ICRS 190.575 32.563333 0.00416666666667,http://archive.stsci.edu/browse/previews/fuse/c0570202/c057020200000specttagf.gif,http://archive.stsci.edu/pub/vospectra/fuse2/c057020200000nvo4ttagfcal_vo.fits,PUBLIC,--,5885.0,science,342901,569779,569779,101.01597069616007
spectrum,2,FUSE,c0570301000,NGC4631-H,190.48333300000002,32.553333,52694.44685185,52694.96520463,22965.0,UV,--,90000000000.0,119000000000.0,--,Observations of O VI Emission in the Halo of NGC 4631,nan,FUV,Murphy,C057,--,--,--,--,CIRCLE ICRS 190.483333 32.553333 0.00416666666667,http://archive.stsci.edu/browse/previews/fuse/c0570301/c057030100000specttagf.gif,http://archive.stsci.edu/pub/vospectra/fuse2/c057030100000nvo4ttagfcal_vo.fits,PUBLIC,--,5885.0,science,342902,569780,569780,149.77766974566998
spectrum,2,FUSE,p1340101000,NGC4631-A,190.53666666699996,32.5766666667,51663.00063657,51663.56815421,21535.0,UV,--,90000000000.0,119000000000.0,--,A Search for OVI Emission in the Halo of NGC 4631,nan,FUV,Murphy,P134,--,--,--,--,CIRCLE ICRS 190.536666667 32.5766666667 0.00416666666667,http://archive.stsci.edu/browse/previews/fuse/p1340101/p134010100000specttagf.gif,http://archive.stsci.edu/pub/vospectra/fuse2/p134010100000nvo4ttagfcal_vo.fits,PUBLIC,--,5885.0,science,345628,572506,572506,45.15588032070208
spectrum,2,FUSE,p1340201000,NGC4631-B,190.53666666699996,32.56,51664.44222222,51664.81761068,16218.0,UV,--,90000000000.0,119000000000.0,--,A Search for OVI Emission in the Halo of NGC 4631,nan,FUV,Murphy,P134,--,--,--,--,CIRCLE ICRS 190.536666667 32.56 0.00416666666667,http://archive.stsci.edu/browse/previews/fuse/p1340201/p134020100000specttagf.gif,http://archive.stsci.edu/pub/vospectra/fuse2/p134020100000nvo4ttagfcal_vo.fits,PUBLIC,--,5885.0,science,345629,572507,572507,0.0
spectrum,2,FUSE,c0570101000,NGC4631-A,190.53666699999997,32.576667,52422.32480324,52422.4911716,7153.0,UV,--,90.0,119.0,--,--,nan,FUV,--,--,--,--,--,--,Circle J2000 190.536667 32.576667 0.00416666666667,http://archive.stsci.edu/missions/fuse/previews/c0570101/c057010100000specttagf.gif,http://archive.stsci.edu/pub/vospectra/fuse2/c057010100000nvo4ttagfcal_vo.fits,PUBLIC,--,5885.0,science,347253,574131,574131,45.15704514422318
spectrum,2,FUSE,c0570401000,NGC4631-I,190.54166699999996,32.551667,52421.42298611,52421.65853036,9407.0,UV,--,90.0,119.0,--,--,nan,FUV,--,--,--,--,--,--,Circle J2000 190.541667 32.551667 0.00416666666667,http://archive.stsci.edu/missions/fuse/previews/c0570401/c057040100000specttagf.gif,http://archive.stsci.edu/pub/vospectra/fuse2/c057040100000nvo4ttagfcal_vo.fits,PUBLIC,--,5885.0,science,347254,574132,574132,18.266360826745768


In [84]:
otb=obs_tables[1]
urls=pl[1]['dataURI']
rr=[x.find('1attagfraw') for x in urls]
dpl=pl[1][np.array(rr)>0]
dpl

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str6,str4,str8,str11,str56,str1,str74,str9,str28,str56,str90,str1,str1,str4,str33,int64,str6,str6,int64
347253,FUSE,spectrum,c0570101000,photon list image : raw,S,mast:FUSE/url/missions/fuse/data/c0570101/c05701010011attagfraw.fit.gz,AUXILIARY,--,--,http://archive.stsci.edu/fuse/DH_Final/Science_Data_Files.html#SECTION00721100000000000000,--,--,--,c05701010011attagfraw.fit.gz,175947,347253,PUBLIC,2
347253,FUSE,spectrum,c0570101000,photon list image : raw,S,mast:FUSE/url/missions/fuse/data/c0570101/c05701010021attagfraw.fit.gz,AUXILIARY,--,--,http://archive.stsci.edu/fuse/DH_Final/Science_Data_Files.html#SECTION00721100000000000000,--,--,--,c05701010021attagfraw.fit.gz,178916,347253,PUBLIC,2
347253,FUSE,spectrum,c0570101000,photon list image : raw,S,mast:FUSE/url/missions/fuse/data/c0570101/c05701010031attagfraw.fit.gz,AUXILIARY,--,--,http://archive.stsci.edu/fuse/DH_Final/Science_Data_Files.html#SECTION00721100000000000000,--,--,--,c05701010031attagfraw.fit.gz,172534,347253,PUBLIC,2
342900,FUSE,spectrum,c0570201000,photon list image : raw,S,mast:FUSE/url/pub/fuse/data/c0570201/c05702010011attagfraw.fit.gz,AUXILIARY,--,--,http://archive.stsci.edu/fuse/DH_Final/Science_Data_Files.html#SECTION00721100000000000000,--,--,C057,c05702010011attagfraw.fit.gz,212492,342900,PUBLIC,2
342900,FUSE,spectrum,c0570201000,photon list image : raw,S,mast:FUSE/url/pub/fuse/data/c0570201/c05702010021attagfraw.fit.gz,AUXILIARY,--,--,http://archive.stsci.edu/fuse/DH_Final/Science_Data_Files.html#SECTION00721100000000000000,--,--,C057,c05702010021attagfraw.fit.gz,224916,342900,PUBLIC,2
342900,FUSE,spectrum,c0570201000,photon list image : raw,S,mast:FUSE/url/pub/fuse/data/c0570201/c05702010031attagfraw.fit.gz,AUXILIARY,--,--,http://archive.stsci.edu/fuse/DH_Final/Science_Data_Files.html#SECTION00721100000000000000,--,--,C057,c05702010031attagfraw.fit.gz,116149,342900,PUBLIC,2
342900,FUSE,spectrum,c0570201000,photon list image : raw,S,mast:FUSE/url/pub/fuse/data/c0570201/c05702010041attagfraw.fit.gz,AUXILIARY,--,--,http://archive.stsci.edu/fuse/DH_Final/Science_Data_Files.html#SECTION00721100000000000000,--,--,C057,c05702010041attagfraw.fit.gz,123528,342900,PUBLIC,2
342900,FUSE,spectrum,c0570201000,photon list image : raw,S,mast:FUSE/url/pub/fuse/data/c0570201/c05702010051attagfraw.fit.gz,AUXILIARY,--,--,http://archive.stsci.edu/fuse/DH_Final/Science_Data_Files.html#SECTION00721100000000000000,--,--,C057,c05702010051attagfraw.fit.gz,133429,342900,PUBLIC,2
342900,FUSE,spectrum,c0570201000,photon list image : raw,S,mast:FUSE/url/pub/fuse/data/c0570201/c05702010061attagfraw.fit.gz,AUXILIARY,--,--,http://archive.stsci.edu/fuse/DH_Final/Science_Data_Files.html#SECTION00721100000000000000,--,--,C057,c05702010061attagfraw.fit.gz,146100,342900,PUBLIC,2
